# Overview

This notebook documents the processing of the raw results from the activation scheme/parcel model sampling data into tidy datasets more amenable for analysis. 

In [2]:
from glob import glob

import os
import pickle

import pandas as pd

The original sampling datasets and their immediate post-processing forms have been copied from `legion` and are archived in **data/{exp_name}**. Additionally, the aerosol sampling datasets have been copied to **data/aerosol_sampling**.

The sampling data is organized into two different sets:

1. **aerosol_design** - Sampling experiment conducted by sub-sampling aerosol/meteorology conditions from MARC output
2. **LHS_design** - Sampling experiment where LHS has been applied to the chaos expansion input parameter spaces.

For the **LHS_design** sampling results, we care about just one file for each scheme,  `data/{scheme_name}/{scheme_name}_sampling_results.csv`, which contains the design as well as all the sampling results. For the **aerosol_design** dataset, we care about four different datasets: 

1. `MARC_aerosol_design.csv` - sampling input parameters, for *both* "main" and "gCCN" modes.
2. `MARC_main_aerosol_design_parameterization_results.csv` - Evaluation of **MARC_main** against aerosol samples, and parameterizatons evaluted using *only* "main" modes.
3. `MARC_gCCN_aerosol_design_parameterization_results.csv` - Same as above, but for **MARC_gCCN**
4. `MARC_gCCN_aerosol_design_parcel_results.csv` - Parcel model evaluations of *full* "gCCN" distribution samples.

--- 

Our objective here is to re-format these datasets for easier visualization and analysis.

# Re-processing

Each of the two data sets is processed separately.

---

## LHS_design

The **LHS_design** results are the evaluation datasets for the two activation schemes. We want to pivot the dataset into a tidy format, where each row is a "sample" result. Each row should encode the following information:

- Scheme name generating sample (main\_*n*, ARG, parcel, etc)
- Variable being analyzed
    - Smax, Neq, Nkn - directly predicted by scheme
    - Nderiv - Droplet number diagnosed from Smax prediction
- Sample id, based on associated **design** table
- Value

In [272]:
from os import environ

if "exp_name" not in environ:
    exp_name = "gCCN"
    print("Manually set exp_name")
else:
    exp_name = environ['exp_name']
orders = [3, 4]

def get_fn(basename, scheme):
    full_path = os.path.join("data/MARC_{scheme}/", basename)
    return full_path.format(scheme=scheme)

sampling_fn = get_fn("MARC_{scheme}_sampling_results.csv", exp_name)
sampling_df = pd.read_csv(sampling_fn, index_col=0)

exp_dict_fn = get_fn("MARC_{scheme}.p", exp_name)
with open(exp_dict_fn, 'rb') as f:
    exp_dict = pickle.load(f)
    
print(sampling_df.columns)

Manually set exp_name
Index(['logN_ACC', 'logN_MOS', 'logN_MBS', 'logN_DST01', 'logN_DST02',
       'logN_SSLT01', 'logmu_ACC', 'logmu_MOS', 'logmu_MBS', 'kappa_MOS',
       'logV', 'T', 'P', 'Smax_parcel', 'Neq_parcel', 'Nkn_parcel',
       'Smax_gCCN_3', 'Nderiv_gCCN_3', 'Neq_gCCN_3', 'Nkn_gCCN_3',
       'Smax_gCCN_4', 'Nderiv_gCCN_4', 'Neq_gCCN_4', 'Nkn_gCCN_4', 'Smax_ARG',
       'Nderiv_ARG', 'Smax_MBN', 'Nderiv_MBN'],
      dtype='object')


Separate the "design" (variables) from the sampling data into their own dataframe. Their index will become the "sample id".

In [117]:
vs = [v[0] for v in exp_dict['variables']]

design_df = sampling_df[vs]
design_df = (
    design_df
    .reset_index()
    .rename(columns={"index": "sample_id"})
    .set_index("sample_id")
)
design_df.head()

,logN_ACC,logN_MOS,logN_MBS,logN_DST01,logN_DST02,logN_SSLT01,logmu_ACC,logmu_MOS,logmu_MBS,kappa_MOS,logV,T,P
sample_id,,,,,,,,,,,,,
0,1.882104,-4.787196,1.279272,0.499861,0.894579,-1.424097,-1.080309,-1.340698,-2.636864,0.583582,0.128252,241.004204,90352.20554
1,-1.228576,3.424074,2.819142,0.176743,0.147766,-1.513440,-0.914070,-1.905798,-2.882728,0.317569,0.725811,308.554350,102805.05600
2,2.663175,-0.909762,0.991011,-4.091173,-3.424029,-0.858034,-0.193682,-1.668761,-2.803899,0.543952,0.796432,245.981379,77480.99490
3,3.719233,3.404198,-3.240300,-0.878332,0.234148,-2.540045,-1.879270,-1.291892,-1.580388,0.148080,0.374067,284.446532,84903.94172
4,3.495877,1.058844,0.179819,0.280961,-1.618518,-2.498890,-2.070286,-2.803842,-2.493860,0.372425,-1.763832,292.536824,50851.32708


We can now manually construct the tidy samplying dataframe.

In [118]:
idx = design_df.index

all_data = []

# Parcel model
parcel_data = (
    # Select just the fields for this scheme
    sampling_df[['Smax_parcel', 'Neq_parcel']]
    # Re-name fields prior to reshaping
    .rename(columns={'Smax_parcel': 'Smax',
                     'Neq_parcel': 'Nderiv'})
    # Set the index to the "sample_id" index from the design,
    # and move to named column
    .set_index(idx).reset_index()
    # Add a column identifying this scheme
    .assign(scheme="parcel")
    # Re-shape into tidy form
    .pipe(pd.melt, id_vars=['scheme', 'sample_id'])
)
all_data.append(parcel_data)

# Parameterizations
scheme_data = []
for scheme in ["ARG", "MBN"]:
    df = (
        sampling_df[['Smax_'+scheme, 'Nderiv_'+scheme]]
        .rename(columns={'Smax_'+scheme: 'Smax',
                         'Nderiv_'+scheme: 'Nderiv'})
        .set_index(idx).reset_index()
        .assign(scheme=scheme)
        .pipe(pd.melt, id_vars=['scheme', 'sample_id'])
    )
    scheme_data.append(df)
all_data.extend(scheme_data)

# Chaos Expansions
chaos_data = []
for scheme in ["{}_{}".format(exp_name, order) for order in orders]:
    df = (
        sampling_df[['Smax_'+scheme, 'Nderiv_'+scheme]]
        .rename(columns={'Smax_'+scheme: 'Smax',
                         'Nderiv_'+scheme: 'Nderiv'})
        .set_index(idx).reset_index()
        .assign(scheme=scheme)
        .pipe(pd.melt, id_vars=['scheme', 'sample_id'])
    )
    chaos_data.append(df)
all_data.extend(chaos_data)

# Concatenate into single DataFrame
all_data = pd.concat(all_data)

Save to an HDF5 file.

In [123]:
hdf = pd.HDFStore('data/MARC_{}_sampling_data.h5'.format(exp_name))
hdf.put("/design", design_df)
hdf.put("/results", all_data)

print(hdf)
hdf.close()

<class 'pandas.io.pytables.HDFStore'>
File path: sampling_data.h5
/gCCN/design             frame        (shape->[10000,13])
/gCCN/results            frame        (shape->[100000,4])
/main/design             frame        (shape->[10000,10])
/main/results            frame        (shape->[100000,4])


---

Sample code for compute statistics over the sample data.

``` python
    subset = all_data.set_index(["variable", "scheme", "sample_id"]).loc['Smax']˘

    ref = subset.loc['parcel']
    groups = subset.groupby(level="scheme")

    import sklearn.metrics as skm
    import numpy as np

    def compute_stats(obs, act):
        """ Create a dictionary with summary statistics comparing two
        datasets. """

        obs = np.asarray(obs)
        act = np.asarray(act)

        mae = skm.mean_absolute_error(act, obs)
        r2 = skm.r2_score(act, obs)
        rmse = np.sqrt(np.sum((obs-act)**2.)/len(act))
        nrmse = rmse/np.sqrt(np.sum((act**2.)/len(act)))

        rel_err = 100.*(obs - act)/act
        # Mask egregiously high values (1000% error) which screw up the spread
        rel_err = rel_err[np.abs(rel_err) <= 1000.]
        mre = np.mean(rel_err)
        mre_std = np.std(rel_err)

        stats = pd.Series({
            'mae': mae, 'r2': r2, 'rmse': rmse, 'nrmse': nrmse,
            'mre': mre, 'mre_std': mre_std,
        })

        return stats


    x = groups.apply(lambda x: compute_stats(ref, x))
```

---

## aerosol_sampling

The **aerosol_sampling** results come from evaluating the activation schemes on sample aerosol distributions from MARC. Again, we want to pivot the dataset into a tidy format, where each row is a "sample" result, such that each row should encode the following information:

- Scheme name generating sample (main\_*n*, ARG, parcel, etc)
- Variable being analyzed
    - Smax, Neq, Nkn - directly predicted by scheme
    - Nderiv - Droplet number diagnosed from Smax prediction
- Sample id, based on associated **design** table
- Value

First we need to load in the tabular data, which is contained in separate
files.

In [5]:
main_data = pd.read_csv("data/aerosol_sampling/MARC_main_aerosol_design_parameterization_results.csv",
                        index_col=0)
gCCN_data = pd.read_csv("data/aerosol_sampling/MARC_gCCN_aerosol_design_parameterization_results.csv",
                        index_col=0)
parcel_data = pd.read_csv("data/aerosol_sampling/MARC_gCCN_aerosol_design_parcel_results.csv",
                          index_col=0)
design_df = (
    pd.read_csv("data/aerosol_sampling/MARC_aerosol_design.csv")
    .reset_index()
    .rename(columns={'index': 'sample_id'})
    .set_index('sample_id')
)

idx = design_df.index

We can process these just like with the previous sampling experiment. The parcel model simulations are straightforward; we'll use the parameterizations from the *gCCN* sampling case.

In [6]:
from itertools import product
all_data = []

# Parcel data
scheme = 'parcel'
parcel_df = (
    parcel_data[['Smax_'+scheme, 'Neq_'+scheme]]
    .rename(columns={'Smax_'+scheme: 'Smax',
                     'Neq_'+scheme: 'Nderiv'})
    .set_index(idx).reset_index()
    .assign(scheme=scheme)
    .pipe(pd.melt, id_vars=['scheme', 'sample_id'])
)
all_data.append(parcel_df)

# Parameterization data
scheme_data = []
for scheme in ["ARG", "MBN"]:
    df = (
        gCCN_data[['Smax_'+scheme, 'Nderiv_'+scheme]]
        .rename(columns={'Smax_'+scheme: 'Smax',
                         'Nderiv_'+scheme: 'Nderiv'})
        .set_index(idx).reset_index()
        .assign(scheme=scheme)
        .pipe(pd.melt, id_vars=['scheme', 'sample_id'])
    )
    scheme_data.append(df)
all_data.extend(scheme_data)

# Chaos Expansions
chaos_data = []
for scheme, order in product(['main', 'gCCN'],
                             [3, 4]):
    df = gCCN_data if scheme == "gCCN" else main_data
    data = (
        df[['Smax_expansion_order_{}'.format(order), 
            'Nderiv_expansion_order_{}'.format(order)]]
        .rename(columns={
                'Smax_expansion_order_{}'.format(order): 'Smax',
                'Nderiv_expansion_order_{}'.format(order): 'Nderiv'
        })
        .set_index(idx).reset_index()
        .assign(scheme="{}_{}".format(scheme, order))
        .pipe(pd.melt, id_vars=['scheme', 'sample_id'])
    )
        
    # There's a small glitch in the output; for the 4th order
    # schemes, a vector of 
    #    [Smax_arg, Nact_arg, Smax_mbn, Nact_mbn, Nderiv]
    # is reported instead of the Nderiv value, so we need
    # to select just that item
    if (order == 4):
        def _sel_last(x):
            x = [float(elem) for elem in x[1:-1].split(",")]
            return x[-1]
        x = data.loc[data.variable == 'Nderiv'].value
        print(order)
        data.loc[data.variable == 'Nderiv', 'value'] = x.apply(_sel_last)
    chaos_data.append(data)
all_data.extend(chaos_data)

# Concatenate into single DataFrame
all_data = pd.concat(all_data)

4
4


Save to an HDF file.

In [19]:
hdf = pd.HDFStore('data/aerosol_data.h5', mode='w')
hdf.put("/design", design_df)
hdf.put("/results", all_data)

print(hdf)
hdf.close()

<class 'pandas.io.pytables.HDFStore'>
File path: data/aerosol_data.h5
/design             frame        (shape->[10000,20])
/results            frame        (shape->[140000,4])


/Users/daniel/anaconda/lib/python3.4/site-packages/IPython/core/interactiveshell.py:2809: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->['scheme', 'variable', 'value']]

  if self.run_code(code, result):


---

Sample code for creating a one-one plot.

``` python
    import numpy as np

    x = hdf['results'].set_index(['variable', 'scheme', 'sample_id'])
    y = x.loc['Smax']
    ref = y.loc['parcel'].value
    test = y.loc['MBN'].value

    df = pd.DataFrame({'ref': ref, 'test': test})
    df[df > 10.] = np.nan
    df[df < -10.] = np.nan
    df = (
        df
        .replace([np.inf, -np.inf], np.nan)
        .dropna()
    ) 

    sns.jointplot('ref', 'test', df, kind='reg')
```